# Shingle Dependency Trees for datasketch's Minhash

In [1]:
import sys
sys.path.append("..")

In [2]:
import conllu
import treesimi as ts
import datasketch
import json
import itertools

## Load Dataset

In [3]:
%%capture
!mkdir "../data"
!wget -O "../data/de_hdt-ud-dev.conllu" "https://raw.githubusercontent.com/UniversalDependencies/UD_German-HDT/master/de_hdt-ud-dev.conllu"

In [4]:
dat = conllu.parse(open("../data/de_hdt-ud-dev.conllu").read())
len(dat)

18434

# Generate Shingles and Build LSH
We are trying the following LSH models available in the datasketch package:

1. `MinHashLSH` -- Index is updatable, Threshold is fixed, Num results unknown (Redis support)
2. `MinHashLSHForest` -- Index is updatable, Threshold unknown, Top-N are returned
3. `MinHashLSHEnsemble` -- Index is fixed, Threshold is fixed (Redis support)

In [5]:
import struct
import mmh3

def hashfunc_mmh3_int32(data: bytes):  # -> packed bytes unsigned 32-bit
    return struct.unpack('<I', struct.pack('<l', mmh3.hash(data)))[0]

_hashfunc = hashfunc_mmh3_int32
# _hashfunc = datasketch.hashfunc.sha1_hash32

In [6]:
%%time

cfg = {
    'use_trunc_leaves': True,
    'use_drop_nodes': False,
    'use_replace_attr': False
}

NUM_PERM = 32

# Instantiate LSH Forest
lsh1 = datasketch.MinHashLSH(num_perm=NUM_PERM, threshold=0.8)
lsh2 = datasketch.MinHashLSHForest(num_perm=NUM_PERM)
lsh3 = datasketch.MinHashLSHEnsemble(num_perm=NUM_PERM, threshold=0.8, num_part=32)
lsh3examples = []

mhash = []
for i in range(len(dat)):
    # Read tree data 
    adjac = [(t['id'], t['head'], t['deprel']) for t in dat[i]]
    nested = ts.adjac_to_nested_with_attr(adjac)
    nested = ts.remove_node_ids(nested)
    
    # Shingling
    shingled = ts.shingleset(nested, **cfg)
    stringified = [json.dumps(tree).encode('utf-8') for tree in shingled]

    # Create MinHash objects
    m = datasketch.MinHash(num_perm=NUM_PERM, hashfunc=_hashfunc)
    for s in stringified:
        m.update(s)

    # Add to LSH
    lsh1.insert(i, m)
    lsh2.add(i, m)
    lsh3examples.append((i, m, len(stringified)))

    # save objects
    mhash.append(m)

# Call index method for LSH Forest
lsh2.index()
# Call index method to build the whole LSH Ensemble
lsh3.index(tuple(lsh3examples))

CPU times: user 52.8 s, sys: 556 ms, total: 53.4 s
Wall time: 54.5 s


# Query the index

## Just LSH

In [7]:
# try to find any clusters of similar sentences
results = []
for j in range(len(mhash)):
    results.append(lsh1.query(mhash[j]))

In [8]:
tmp = []
for i, res in enumerate(results):
    # number of examples with at least 1 similar example
    if len(res) >= 3 and len(res) <= 10:
        # mininum number of tokens
        if len(dat[i]) >= 8:
            tmp.append(res)

print(len(set(itertools.chain(*tmp))))

85


In [12]:
res = tmp[0]
j = res[0]
# print(j)

for i in res:
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text'][:70]}")

   17409 | 1.0000 | Dann sei das Projekt aber durch Patentstreitigkeiten bedroht .
   15748 | 0.6562 | In Kalifornien sollen in Zukunft die Bürger via Internet vor Erdbeben 
     205 | 0.7812 | " Das Dokument kann ohne Werbung nicht dargestellt werden .


## Top-N Results (LSH Forest)

In [10]:
j = 5000
res = lsh2.query(mhash[j], 5)
print(f"#num results: {len(res)}")

for i in res:
    print(f"{i:>8d} | {mhash[i].jaccard(mhash[j]):6.4f} | {dat[i].metadata['text'][:70]}")

#num results: 5
     352 | 0.2188 | Fünf Online-Unterrichtseinheiten sollen unter ab dem 15. September unt
    5000 | 1.0000 | 25 Bilder pro Sekunde in voller PAL-Auflösung lassen sich laut Formac 
      40 | 0.2500 | In Hamburg werden daher derzeit zwei Modelle für einen Studententarif 
     308 | 0.2500 | Diejenigen , bei denen die Einwahl nach etlichen Versuchen geklappt ha
     221 | 0.1875 | Zwei Hacker hatten der Redaktion vorgeführt , daß plumpe Täuschungsman


## Containment Query (LSH Ensemble)

In [11]:
j = 5000
res = [key for key in lsh3.query(lsh3examples[j][1], lsh3examples[j][2])]
print(f"#num results: {len(res)}")

for i in res:
    jacc = mhash[i].jaccard(mhash[j])
    if jacc > 0.35:
        print(f"{i:>8d} | {jacc:6.4f} | {dat[i].metadata['text'][:70]}")

#num results: 312
   12620 | 0.3750 | Nach Herstellerangaben arbeiten derzeit 1,5 Millionen Anwender mit Ope
    9332 | 0.4375 | Neben Siemens wurden weitere sieben Firmen wegen Leistungsmängeln bei 
   17527 | 0.3750 | Die Telematik-Sparte von Motorola hat zwei dicke Fische an Land gezoge
    5000 | 1.0000 | 25 Bilder pro Sekunde in voller PAL-Auflösung lassen sich laut Formac 
   14498 | 0.3750 | Die AGP-Karte mit TNT2-Grafikbeschleuniger verfügt über 32 MByte Bilds
   18214 | 0.4375 | Drei Anbieter von sogenannten Pet-Portals streiten sich um den mit Pag
    5734 | 0.4688 | Um Abhilfe zu schaffen , werde Comdirect demnächst ein neues Service-C
    1667 | 0.3750 | Seit dem 12. April kostet der Zugang zum Internet bei 1&1 abends 4 Pfe
    3342 | 0.3750 | Laut Hersteller bietet EPOC-Opera ferner eine 128 Bit starke Verschlüs
    3693 | 0.3750 | Der Dual-CPU-Mac mit 800er CPUs kostet momentan in den USA 3500 Dollar
   13023 | 0.3750 | Somit entspricht ein manipuliertes 30-Tage-Paket noc